In [ ]:
# Native libraries
from tqdm.notebook import tqdm

# third-party libraries
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from InstructorEmbedding import INSTRUCTOR

c:\Users\min hua\miniconda3\envs\pytorch_310_hf\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
# inputs
model_name = 'hkunlp/instructor-large'

# Set similarity threshold (after experimenting, 0.85 seems good)
threshold = 0.77

input_file_dir = r'common_voice/cv-valid-dev-q4.csv'
output_file_dir = r'common_voice/cv-valid-dev-q5.csv'

# Define hot words
hot_words = ["be careful", "destroy", "stranger"]

In [23]:

# Load model
model = INSTRUCTOR(model_name)

# prepare hot words' embedding
hot_word_embeddings = {}
instruction = "Represent the text for semantic similarity:"
for word in hot_words:
    hot_word_embeddings[word] = model.encode([[instruction, word]])

# prepare similarity boolean array
similarity_bool_arr = []

# Load csv file to run through every text
df = pd.read_csv(input_file_dir)
for _, row in tqdm(df.iterrows()):
    # Clean and split text into words
    input_text = str(row['text']).lower()

    # prepare input text's embeddings
    input_text_embedding = model.encode([[instruction, input_text]])
    
    # Check similarity between each hot word and input text embeddings
    for hot_word, hot_word_embedding in hot_word_embeddings.items():
        similarity = cosine_similarity(
            input_text_embedding.reshape(1, -1),
            hot_word_embedding.reshape(1, -1)
        )[0][0]
        
    # condition for similarity column; if more than threshold, means very similar
    if similarity > threshold:
        similarity_bool_arr.append(True)
    else:
        similarity_bool_arr.append(False)


# Save results
df['similarity'] = similarity_bool_arr
df.to_csv(output_file_dir, index=False)

load INSTRUCTOR_Transformer
max_seq_length  512


0it [00:00, ?it/s]

In [ ]:
# # # Use this test script to test for the threshold
# # # 3006, 1498, 900
# input_text = str(df.iloc[900]['text']).lower()

# # prepare input text's embeddings
# input_text_embedding = model.encode([[instruction, input_text]])

# # Check similarity between each hot word and input text embeddings
# for hot_word, hot_word_embedding in hot_word_embeddings.items():
#     similarity = cosine_similarity(
#         input_text_embedding.reshape(1, -1),
#         hot_word_embedding.reshape(1, -1)
#     )[0][0]

# similarity

0.7751745